In [1]:
from fastai.text import *
from fastai.lm_rnn import *
import sentencepiece as sp
from scipy import stats
import fastai.core

In [2]:
fastai.core.USE_GPU = True

In [3]:
torch.cuda.set_device(2)

In [4]:
BTW17 = Path("../data/btw17")
GE17  = Path("../data/germeval2017")
WORK = Path("../work/")
WIKI=Path("../data/wiki/de")
!ls {WORK}
!ls {GE2017}/
!ls {WIKI}

btw-nouniq30k  ge2017  shared  shared-wiki  wiki30k  wikige2017
ls: cannot access '{GE2017}/': No such file or directory
train.csv  train.txt  val.csv


In [5]:
GE17WRK = WORK/"ge2017/tmp"
GE17ML = WORK/"ge2017/models"
!ls {GE17WRK}

dev_ids.npy    lbl_test2.npy  sp.model	     trn_ids_all.npy
lbl_dev.npy    lbl_trn.npy    test1_ids.npy  trn_ids.npy
lbl_test1.npy  lbl_val.npy    test2_ids.npy  val_ids.npy


In [13]:
lbl = np.load(GE17WRK/'lbl_val.npy')

In [14]:
inp = np.load(GE17WRK/'trn_ids.npy')

In [15]:
lbl.shape, inp.shape

((2584,), (20941,))

In [18]:
lbl[0], inp[0]

(1,
 array([    2,     4,  2796,    11,  4367,   171,    12,   394,    24, 17694,  2864,  8451,    71,    12,
         7979,   345,    83,    61,    12,   126,  5460,    96,   134,  4310,    23, 17500,   152,    27,
            3]))

In [19]:
f = GE17WRK/'lbl_val.npy'

In [34]:
f.exists()

True

In [48]:
def mk_bwd(f):
    if f.suffix != '.npy':
        print("Ingoring", f, 'not a npy')
        return
    if not f.stem.endswith('_ids'):
        print(f'Ignoring {f}, does not ends with _ids')
        return
    n = f.with_name(str(f.name).replace('.npy', '_bwd.npy'))
    if n.exists():
        print("Backward version exist, ignoring",f)
        return
    print(f"Reversing {f} -> {n}")
    arr = np.load(f)
    res = np.array([np.flip(a, axis=0) for a in arr])
    
    np.save(str(n), res)

In [50]:
for p in WORK.glob('**/*.npy'): mk_bwd(p)

Reversing ../work/wiki30k/tmp/val_ids.npy -> ../work/wiki30k/tmp/val_ids_bwd.npy
Reversing ../work/wiki30k/tmp/trn_ids.npy -> ../work/wiki30k/tmp/trn_ids_bwd.npy
Reversing ../work/wikige2017/tmp/test2_ids.npy -> ../work/wikige2017/tmp/test2_ids_bwd.npy
Ignoring ../work/wikige2017/tmp/lbl_test1.npy, does not ends with _ids
Reversing ../work/wikige2017/tmp/val_ids.npy -> ../work/wikige2017/tmp/val_ids_bwd.npy
Ignoring ../work/wikige2017/tmp/trn_ids_all.npy, does not ends with _ids
Ignoring ../work/wikige2017/tmp/lbl_trn.npy, does not ends with _ids
Reversing ../work/wikige2017/tmp/test1_ids.npy -> ../work/wikige2017/tmp/test1_ids_bwd.npy
Ignoring ../work/wikige2017/tmp/lbl_test2.npy, does not ends with _ids
Reversing ../work/wikige2017/tmp/trn_ids.npy -> ../work/wikige2017/tmp/trn_ids_bwd.npy
Ignoring ../work/wikige2017/tmp/lbl_val.npy, does not ends with _ids
Reversing ../work/btw-nouniq30k/tmp/val_ids.npy -> ../work/btw-nouniq30k/tmp/val_ids_bwd.npy
Reversing ../work/btw-nouniq30k/tmp/

In [54]:
for p in sorted(WORK.glob('**/*.npy')): print(p)

../work/btw-nouniq30k/tmp/trn_ids.npy
../work/btw-nouniq30k/tmp/trn_ids_bwd.npy
../work/btw-nouniq30k/tmp/val_ids.npy
../work/btw-nouniq30k/tmp/val_ids_bwd.npy
../work/ge2017/tmp/lbl_dev.npy
../work/ge2017/tmp/lbl_test1.npy
../work/ge2017/tmp/lbl_test2.npy
../work/ge2017/tmp/lbl_trn.npy
../work/ge2017/tmp/lbl_val.npy
../work/ge2017/tmp/test1_ids.npy
../work/ge2017/tmp/test1_ids_bwd.npy
../work/ge2017/tmp/test2_ids.npy
../work/ge2017/tmp/test2_ids_bwd.npy
../work/ge2017/tmp/trn_ids.npy
../work/ge2017/tmp/trn_ids_all.npy
../work/ge2017/tmp/trn_ids_bwd.npy
../work/ge2017/tmp/val_ids.npy
../work/ge2017/tmp/val_ids_bwd.npy
../work/wiki30k/tmp/trn_ids.npy
../work/wiki30k/tmp/trn_ids_bwd.npy
../work/wiki30k/tmp/val_ids.npy
../work/wiki30k/tmp/val_ids_bwd.npy
../work/wikige2017/tmp/lbl_test1.npy
../work/wikige2017/tmp/lbl_test2.npy
../work/wikige2017/tmp/lbl_trn.npy
../work/wikige2017/tmp/lbl_val.npy
../work/wikige2017/tmp/test1_ids.npy
../work/wikige2017/tmp/test1_ids_bwd.npy
../work/wikige20